A class to allow global analysis of stress granules labeled with Riboglow.

In [2]:
%matplotlib inline

In [6]:
%%writefile SgFinder.py

from nd2reader import ND2Reader
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import numpy as np
import seaborn as sn
import sys
from tifffile import TiffFile

class SgFinder:
    def __init__(self, im_path, rG_background, haloTag_channel=2, rG_channel=1):
        if im_path[-4:] == '.nd2':
            self.image = []
            with ND2Reader(im_path) as nd2:
                for channel in range(3):
                    self.image.append(nd2[channel])
        elif im_path[-4:] == '.tif':
            with TiffFile(im_path) as tif:
                self.image = tif.asarray()
        else:
            print("check yo extension. I found '{}'. I can only use '.nd2' or '.tif'.".format(im_path[-4:]))
        self.haloTag_channel = haloTag_channel
        self.rG_channel = rG_channel
        self.rG_background = rG_background
        self.haloTag_threshold = None
        self.granule_count = None
        self.granule_mask = None
        self.ratios = []
    
    def median_filter(self, pixels):
        for channel in range(3):
            self.image[channel] = ndi.median_filter(self.image[channel], size=pixels)
    
    def setHaloThreshold(self, threshold=None, slider_range=(0,3000)):
        
        def t(threshold_value):
            t_img = self.image[self.haloTag_channel] > threshold_value
            fig, ax = plt.subplots(ncols=2,figsize=(10, 10))
            ax[0].imshow(self.image[self.haloTag_channel])
            ax[1].imshow(t_img)
            #plt.imshow(t_img)
            plt.show()
            labeled_sg, count = ndi.label(self.image[self.haloTag_channel]>threshold_value)
            print("found {}".format(count))
            self.haloTag_threshold = threshold_value
            self.granule_count = count
            self.granule_mask = labeled_sg
            
        if threshold is not None:
            self.haloTag_threshold = threshold
            slider_value = threshold
            t(threshold)
        else:
            slider_value = 0
#         def t(threshold_value):
#             t_img = self.image[self.haloTag_channel] > threshold_value
#             fig, ax = plt.subplots(figsize=(10, 10))
#             ax.imshow(t_img)
#             #plt.imshow(t_img)
#             plt.show()
#             labeled_sg, count = ndi.label(self.image[self.haloTag_channel]>threshold_value)
#             print("found {}".format(count))
#             self.haloTag_threshold = threshold_value
#             self.granule_count = count
#             self.granule_mask = labeled_sg

            slider = widgets.IntSlider(min=slider_range[0], max=slider_range[1], step=10, value=slider_value)
            interact(t, threshold_value=slider)
        
    def dilate_and_ratio(self, iterations=3, sig_noise_requirement=3):
        self.ratios = []
        for label in range(1,self.granule_count):
            print("completed {} of {}".format(label, self.granule_count), end="\r")
            mask = self.granule_mask==label
            rg_sg_mask = np.ma.array(self.image[self.rG_channel], mask=np.invert(mask))
            sg_median = np.ma.median(rg_sg_mask)
            if sg_median<sig_noise_requirement*self.rG_background: continue
            dilation = ndi.morphology.binary_dilation(mask, iterations=iterations) # might have to adjust iterations
            outline = np.logical_xor(dilation, mask)
#            rg_sg_mask = np.ma.array(self.image[self.rG_channel], mask=np.invert(mask))
            rg_outline_mask = np.ma.array(self.image[self.rG_channel], mask=np.invert(outline))
#            sg_median = np.ma.median(rg_sg_mask)
            outline_median = np.ma.median(rg_outline_mask)
            if (sg_median<sig_noise_requirement*self.rG_background) or (outline_median<sig_noise_requirement*self.rG_background): continue
            self.ratios.append(sg_median/outline_median)
        
        print("found {} loaded particles. \n Average Ratio: {}".format(len(self.ratios), np.mean(self.ratios)))

Overwriting batch_SG_analysis.py


In [147]:
test = SgFinder('./20180831_A1_5.nd2', 82)
test.median_filter(1)

In [148]:
test.setHaloThreshold()

interactive(children=(IntSlider(value=0, description='threshold_value', max=3000, step=10), Output()), _dom_cl…

In [149]:
test.dilate_and_ratio()

found 42 loaded particles. 
 Average Ratio: 1.310614543864754


In [151]:
testneg = SgFinder('./20180831_neg1_6_2.nd2', 36)
testneg.median_filter(1)
testneg.setHaloThreshold(threshold=700)

interactive(children=(IntSlider(value=700, description='threshold_value', max=3000, step=10), Output()), _dom_…

In [152]:
testneg.dilate_and_ratio()

found 45 loaded particles. 
 Average Ratio: 1.157799009828825


In [62]:
test_tiff = SgFinder('20180917_A_Cy_large1_crop.tif', 34, rG_channel=2, haloTag_channel=1)

In [63]:
test_tiff.setHaloThreshold()

interactive(children=(IntSlider(value=0, description='threshold_value', max=3000, step=10), Output()), _dom_cl…

In [64]:
test_tiff.dilate_and_ratio()

found 38 loaded particles. 
 Average Ratio: 1.360826289098006


In [137]:
test_large = SgFinder('20180917_A_Cy_large1.tif', rG_background=34, rG_channel=2, haloTag_channel=1)
test_large.median_filter(2)

In [138]:
test_large.setHaloThreshold(threshold=470)
#test_large.setHaloThreshold()

interactive(children=(IntSlider(value=470, description='threshold_value', max=3000, step=10), Output()), _dom_…

In [139]:
test_large.dilate_and_ratio()

found 68 loaded particles. 
 Average Ratio: 1.4572437466299828


In [140]:
test_neg_large = SgFinder('20180917_neg_Cy_large4t.tif', 25, rG_channel=2, haloTag_channel=1)

In [141]:
test_neg_large.median_filter(2)

In [145]:
test_neg_large.setHaloThreshold(threshold=1000)

interactive(children=(IntSlider(value=1000, description='threshold_value', max=3000, step=10), Output()), _dom…

In [146]:
test_neg_large.dilate_and_ratio()

found 98 loaded particles. 
 Average Ratio: 1.0768230398915097
